In [2]:
"""
transactions.ipynb

Goal with this notebook is to extend the analysis of blocks.ipynb
to transactions within each block for more detail.
"""

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import typing as tp

from ape import accounts, chain, Contract, networks
from scipy import stats

In [3]:
# SEE: https://gist.github.com/banteg/dcf6082ff7fc6ad51ce220146f29d9ff
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

In [5]:
# look at the current block
current_block = chain.blocks[-1]
current_block

Block(num_transactions=111, hash=HexBytes('0x78531930c5d099842e4db1acfccdeabb4e52837654722b9be33bba65457783e4'), number=15775055, parent_hash=HexBytes('0x68c76e401c5b27c995330769e5602b2bfc061d89a87f7fa079c9fb99ccbfc8e8'), size=54872, timestamp=1666096895, gas_limit=30000000, gas_used=10508242, base_fee=13916923898, difficulty=0, total_difficulty=58750003716598352816469)

In [6]:
# some relevant numbers
start_block_number = 15338009 # ~ 1 month before merge
merge_block_number = 15537394
current_block_number = current_block.number

In [7]:
# load EL blocks from csv if already queried, else do the query and save in csv
# SEE: blocks.ipynb
def load_blocks(fp: str) -> pd.DataFrame:
    if os.path.exists(fp):
        return pd.read_csv(fp)
    else:
        qb = chain.blocks.query('*', start_block=start_block_number)
        qb.to_csv(fp, index=False)
        return qb

In [8]:
# get blocks over last 2 months (since mid Aug to mid October)
# NOTE: query takes around ~15h if not from csv
%time qb = load_blocks('./data/blocks.csv')

CPU times: user 921 ms, sys: 84.9 ms, total: 1.01 s
Wall time: 1.04 s


In [9]:
qb

,num_transactions,hash,number,parent_hash,size,timestamp,gas_limit,gas_used,base_fee,difficulty,total_difficulty
0,97,"b'\xf0\x82\x95\x1e@\xe4by\xfc,\x15\xd6\xb6xFw\...",15338009,b'm\xb0\x02|\xa9}\x84\xa3\xdd\xc1\x84\xb6\xe4\...,39308,1660456780,30000000,9954560,8122412901,12045723921070914,56321843715293942409414
1,305,b'\xdb\x91:\xfc8Qk\xc1c\x82\xe2\xdbA\xe9b#R\xe...,15338010,"b'\xf0\x82\x95\x1e@\xe4by\xfc,\x15\xd6\xb6xFw\...",99210,1660456787,30000000,22464133,7780903344,12051743061157721,56321855767037003567135
2,51,b'\xb0\x0c2\x7f$\xc0\xec$i\xbf\xd9\xcb\x13\x0c...,15338011,b'\xdb\x91:\xfc8Qk\xc1c\x82\xe2\xdbA\xe9b#R\xe...,13806,1660456811,30000000,3781159,8264884155,12045995859944613,56321867813032863511748
3,24,"b""0\xbc\xf0\xa6\x89\xf1w\x1d\x9d\x05\x1c\xc8\x...",15338012,b'\xb0\x0c2\x7f$\xc0\xec$i\xbf\xd9\xcb\x13\x0c...,7824,1660456814,29970705,1551283,7492197312,12057896966730061,56321879870929830241809
4,63,b'`/\xd0\xe0\x7fvc>\x94\xf2\xd1\xca\xbf\x11\x9...,15338013,"b""0\xbc\xf0\xa6\x89\xf1w\x1d\x9d\x05\x1c\xc8\x...",16003,1660456819,29941438,3174533,6652621642,12063922050686819,56321891934851880928628
...,...,...,...,...,...,...,...,...,...,...,...
416045,189,b'\xecP\xf7g\x08\xc2\x19\x12_\x0c\xd8X\x92\x9b...,15754054,b'\x12Y[\x91y.\xb8\xcf\xd7\xca\xb5\x81\x87\x82...,82178,1665843383,30000000,18028304,14001417244,0,58750003716598352816469
416046,253,b'n\xf2R{ \xa9t\xabUD\xbc\x8e\x04B\x87\xb3\t\x...,15754055,b'\xecP\xf7g\x08\xc2\x19\x12_\x0c\xd8X\x92\x9b...,159816,1665843395,30000000,29936690,14354755142,0,58750003716598352816469
416047,137,b'\xb7\x06\xba9s\x7f>p\xd1\xa3\x16\xb0)Y\n\xd4...,15754056,b'n\xf2R{ \xa9t\xabUD\xbc\x8e\x04B\x87\xb3\t\x...,71368,1665843407,30000000,14425421,16141526205,0,58750003716598352816469
416048,117,b'\x86\xd9n\xfb+\x9e\n\xbc\xb3\x99\x155@\xbb\x...,15754057,b'\xb7\x06\xba9s\x7f>p\xd1\xa3\x16\xb0)Y\n\xd4...,56671,1665843419,30000000,11198548,16064238022,0,58750003716598352816469


In [90]:
from functools import partial
from hexbytes import HexBytes
from typing import List

from ape.api.providers import BlockAPI, TransactionAPI
from ape.api.query import BlockTransactionQuery, extract_fields, validate_and_expand_columns

# for each block in blocks query, query for transactions
def transaction_container_query(block, *columns: List[str]) -> pd.DataFrame:
    """
    Implements what could be a transaction "container" query analogous
    to https://github.com/ApeWorX/ape/blob/main/src/ape/managers/chain.py#L94
    but for transactions.
    """
    # perform BlockTransactionQuery
    # SEE: https://github.com/ApeWorX/ape/blob/main/src/ape/api/providers.py#L92
    query = BlockTransactionQuery(columns=columns, block_id=int(block.number))
    transactions = chain.query_manager.query(query) # use chain here so block can be row in pd.DataFrame
    
    # put into a dataframe and return
    columns = validate_and_expand_columns(columns, TransactionAPI) # NOTE: this might not include all fields we want!
    transactions = map(partial(extract_fields, columns=columns), transactions)
    df = pd.DataFrame(columns=columns, data=transactions)
    
    # add in columns for block number and block hash then return
    df['block_hash'] = [ block.hash for i in range(len(df)) ]
    df['block_number'] = [ block.number for i in range(len(df)) ]
    return df

In [92]:
# :) works
print('qb.iloc[0]', qb.iloc[0])
transaction_container_query(qb.iloc[0], '*')

qb.iloc[0] num_transactions                                                   97
hash                b'\xf0\x82\x95\x1e@\xe4by\xfc,\x15\xd6\xb6xFw\...
number                                                       15338009
parent_hash         b'm\xb0\x02|\xa9}\x84\xa3\xdd\xc1\x84\xb6\xe4\...
size                                                            39308
timestamp                                                  1660456780
gas_limit                                                    30000000
gas_used                                                      9954560
base_fee                                                   8122412901
difficulty                                          12045723921070914
total_difficulty                              56321843715293942409414
Name: 0, dtype: object


,chain_id,receiver,sender,gas_limit,nonce,value,data,type,max_fee,max_priority_fee,required_confirmations,signature,block_hash,block_number
0,1,0x70526cc7a6d6320B44122ea9d2d07670ACcC85A1,0xa00CcE999f4cdde0230360609ed94e3b64b09EC1,290156,24042,0,b'0x00c1b683cc10030f000000000000221666b3fc9534...,0x02,10282974732,2.500000e+09,7,<TransactionSignature v=0 r=2e7b2dff43593be420...,"b'\xf0\x82\x95\x1e@\xe4by\xfc,\x15\xd6\xb6xFw\...",15338009
1,1,0xdb7A53E6AE058E1Dcf4502341E2ADFA522E2B29F,0x8E17d7A5Eeb39E558021e0B5F75Ad60CF9a1a939,253902,4295,0,b'0x0618d8033dfcad6a002265fcf39374ae0003150525...,0x00,0,NaN,7,<TransactionSignature v=37 r=0bf26aa00960a2738...,"b'\xf0\x82\x95\x1e@\xe4by\xfc,\x15\xd6\xb6xFw\...",15338009
2,1,0xd5FBDa4C79F38920159fE5f22DF9655FDe292d47,0x541a46FFf74cAEFD2a54B752c48b02Ba9864d51E,21000,191,69925781296602000,b'0x',0x02,26210890638,2.621089e+10,7,<TransactionSignature v=0 r=de2d14fc2824b99aa6...,"b'\xf0\x82\x95\x1e@\xe4by\xfc,\x15\xd6\xb6xFw\...",15338009
3,1,0xd5FBDa4C79F38920159fE5f22DF9655FDe292d47,0x541a46FFf74cAEFD2a54B752c48b02Ba9864d51E,21000,192,69943042248093000,b'0x',0x02,25388940567,2.538894e+10,7,<TransactionSignature v=0 r=a93837dc4bb3c0c894...,"b'\xf0\x82\x95\x1e@\xe4by\xfc,\x15\xd6\xb6xFw\...",15338009
4,1,0x1111111254fb6c44bAC0beD2854e76F90643097d,0x0e3Fbeba7c45666F2EcacEE7c60844a9C1844151,460110,73,14540000000000000000,b'0x7c025200000000000000000000000000288931fa76...,0x02,14720610463,7.500000e+09,7,<TransactionSignature v=0 r=754ff55d40bf584814...,"b'\xf0\x82\x95\x1e@\xe4by\xfc,\x15\xd6\xb6xFw\...",15338009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,1,0xb0d47dD82fb8FACb1Bc4bA534a836B545aD97d2B,0x39B984d1D427BD74a7713537A27530d2963B92B2,142195,120,0,b'0xa9059cbb00000000000000000000000073bf7d4edd...,0x02,8703000983,1.000000e+09,7,<TransactionSignature v=1 r=7bde3123c97e5c06c2...,"b'\xf0\x82\x95\x1e@\xe4by\xfc,\x15\xd6\xb6xFw\...",15338009
93,1,0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45,0x385d94dE0BE5F6d958B8340a8BE6F51c4D53bD87,216907,119,50000000000000000,b'0x5ae401dc0000000000000000000000000000000000...,0x02,8703000983,1.000000e+09,7,<TransactionSignature v=0 r=84dea5997f56e3cb49...,"b'\xf0\x82\x95\x1e@\xe4by\xfc,\x15\xd6\xb6xFw\...",15338009
94,1,0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45,0xD72276Ef7305Be51C1FdaFCB571B274B55e98526,349912,514,20000000000000000,b'0x5ae401dc0000000000000000000000000000000000...,0x02,8703000983,1.000000e+09,7,<TransactionSignature v=1 r=bfc5278e179af89505...,"b'\xf0\x82\x95\x1e@\xe4by\xfc,\x15\xd6\xb6xFw\...",15338009
95,1,0x65c2e54a4c75ff6da7b6B32369c1677250075fb2,0x87619f8e63cD2F339f1EBD5144fD3fE37871c204,177495,0,7514540636130913,b'0x7649b9570000000000000000000000000000000000...,0x02,8703000983,1.000000e+09,7,<TransactionSignature v=0 r=5991e716eeec68d5f5...,"b'\xf0\x82\x95\x1e@\xe4by\xfc,\x15\xd6\xb6xFw\...",15338009


In [108]:
# load EL transactions from csv if already queried, else do the query and save in csv
def load_transactions(fp: str, qb: pd.DataFrame) -> pd.DataFrame:
    if os.path.exists(fp):
        return pd.read_csv(fp)
    else:
        # query transactions in each block
        # NOTE: qt is of type pd.Series[pd.DataFrame]
        qt = qb.apply(lambda b: transaction_container_query(b, '*'), axis=1)
        
        # flatten into dataframe
        qt = pd.concat([v for _, v in qt.items()])
        
        # save to csv and return
        qt.to_csv(fp, index=False)
        return qt

In [ ]:
# get txs in each block over last 2 months (since mid Aug to mid October)
# NOTE: query takes around ~X h if not from csv
%time qt = load_transactions('./data/transactions.csv', qb)